In [1]:
import pandas as pd
import mlxtend
from mlxtend.frequent_patterns import association_rules, apriori
import seaborn as sd
import numpy as np
import matplotlib.pyplot as plt

#                      project data analyst Market Basket Analysis

- 1 โปรเจ็คนี้เกิดขึ้นเพราะความยากรู้ของผู้ที่ทำการศึกษาข้อมูลมีความสงสัยในรายงานการขายสินค้าของบริษัทตัวเองว่า ลูกค้าที่ไม่เข้ารับการบริการจากศูนย์บริการรถยนต์เข้าชื้ออะไหล่ประเภทไหน และ ผู้ชื้อเป็นลูกค้าประเภทไหน ?
- 2 เพื่อความสงสัยเลยต้องการใช้ทักษะด้าน data analystเข้ามา วิเคราะห์ข้อมูลว่า สินค้าที่ลูกค้าชื้อ มีอะไรบาง โดยใช้ วิธี Market Basket Analysis ในการหาคำตอบของคำถาม

# ขั้นตอนที่ 1 import data(ข้อมูลเป็นของปี 2022)
- ข้อมูลบิลเงินสดในการขายสินค้าหน้าร้านของการขายสินค้าประเภทอะไหล่รถยนต์ในปี 2023

In [2]:
data = pd.read_excel('ฝึกวิเคราะห์ตระกร้าสินค้าลูกค้า.xlsx')
data[:5]

,date,bill_number,costomer_code,part_name,quantity
0,2022-01-03,A22-00001,4,น้ำยาหม้อน้ำลิตร,1
1,2022-01-03,A22-00001,4,ยางปัดน้ำฝนหลัง,1
2,2022-01-04,A22-00002,4209,ถ่านกุญแจรีโมท,1
3,2022-01-04,A22-00003,4309,ลูกยางสวิชไฟเบคร,1
4,2022-01-04,A22-00004,4,น้ำยาหม้อน้ำแกลลอน,1


# ทำการดูข้อมูลโดยการเรียงลำดับจากมากไปหาน้อย
- เพื่อทำการดูข้อมูลเบื้องต้นว่า สินค้าอะไรที่มีลูกค้าชื้อในจำนวนเยอะๆ
- ผลปราฏว่า สินค้าที่ลูกค้า ชื้อในปริมาณที่เยอะก็คือ สินค้าประเภท หมุดยึดต่างๆ เป็นอะไหล่ กิ่งมาตราฐาน
- โดยจะทำการดึงข้อมูลประมาณ 30 ตัว แรกที่ขายได้บ่อยๆ

In [3]:
data = data.sort_values("quantity", ascending = False).head(30)
data

,date,bill_number,costomer_code,part_name,quantity
419,2022-09-19,A22-00314,4252,หมุดยึด,8
260,2022-06-22,A22-00199,5,ฝาปิดสเกิร์ต,8
492,2022-10-18,A22-00357,4212,หมุดยึด,7
409,2022-09-12,A22-00304,4221,หมุดยึด,7
454,2022-10-05,A22-00336,4,หมุดยึดกันชน,6
211,2022-05-21,A22-00160,4442,หมุดยึดกันชน,5
558,2022-11-25,A22-00409,8,คลิ๊บแผงใต้เครื่อง,5
213,2022-05-23,A22-00161,4442,หมุดยึดกันชน,5
286,2022-07-04,A22-00214,4009,บู๊ชหูแหนบ,4
623,2022-12-23,A22-00452,2,หัวเทียน,4


# ดูสถิติเบื้องต้นของการชื้อสินค้าหน้าร้าน
- จากการดูสินค้าที่ขายหน้าร้านพบว่า
- part_name ของสินค้าที่ขายจริงๆก็พบว่ามีแค่ 30อย่าง เท่านั้นที่ขายได้บ๋อยที่สุด
- top part_nameของสินค้าที่ขายได้คือ หมุดยึด ที่มีข้อมูลความถี่ที่ 8 ตัว
- mean ค่าเฉลี่ยที่ชื้ออยู่ที่ประมาณ 4.43 ชิ้นต่อบิล
- max สูงสุด 8 ชิ้น

In [4]:
data.describe(include='all')

C:\Users\RACHAPAK\AppData\Local\Temp\ipykernel_10084\1336221646.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data.describe(include='all')


,date,bill_number,costomer_code,part_name,quantity
count,30,30,30.000000,30,30.000000
unique,26,28,NaN,14,NaN
top,2022-12-29 00:00:00,A22-00461,NaN,หมุดยึด,NaN
freq,2,2,NaN,8,NaN
first,2022-01-18 00:00:00,NaN,NaN,NaN,NaN
last,2022-12-29 00:00:00,NaN,NaN,NaN,NaN
mean,NaN,NaN,2259.566667,NaN,4.433333
std,NaN,NaN,2147.660229,NaN,1.406471
min,NaN,NaN,2.000000,NaN,3.000000
25%,NaN,NaN,5.500000,NaN,4.000000


# หลังจากที่ทราบแล้วว่า สถิติเป็นอย่างไรก็มาทำการ จัดกลุ่มข้อมูล
- จัดกลุ่มข้อมูลเพื่อให้อยู่ในรูปแบบที่เหมาะสมสำหรับการคำนวณ Market Basket Analysis

In [5]:
basket = (data.groupby(['bill_number','part_name'])['quantity'].sum()\
.unstack().reset_index().fillna(0).set_index('bill_number'))
basket

part_name,คลิ๊บยางรองกันฝุ่น,คลิ๊บล็อค,คลิ๊บแผงใต้เครื่อง,น๊อต ยึดหน้าแกน,บู๊ชหูแหนบ,ประเก็นหัวเทียน,ฝาครอบกระจกมองข้างขวา,ฝาปิดสเกิร์ต,พลาสติกรองสกรู,ยางกันฝุ่นเบคร,สกรู,หมุดยึด,หมุดยึดกันชน,หัวเทียน
bill_number,,,,,,,,,,,,,,
A22-00024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
A22-00033,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A22-00034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
A22-00052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
A22-00120,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A22-00138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
A22-00160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
A22-00161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
A22-00165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0


# ทำการแปลงข้อมูลสินค้า
- เนื่องจากสินค้ามีจำนวนมากและหลากหลายและข้อมูลเยอะ เนื่องจากลูกค้าอาจจะชื้อสินค้าหลายชิ้น
- อาจจะทำให้คำนวณได้ยาก จึงต้องทำการแปลงข้อมูล จำนวนการชื้อสินค้าก่อน
- เพื่อให้ง่ายต่อการคำนวณ

In [6]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)

# ทำการสำรวจข้อมูลดูว่าแปลงตามที่ต้องการหรือไม่ ?

In [7]:
basket_sets

part_name,คลิ๊บยางรองกันฝุ่น,คลิ๊บล็อค,คลิ๊บแผงใต้เครื่อง,น๊อต ยึดหน้าแกน,บู๊ชหูแหนบ,ประเก็นหัวเทียน,ฝาครอบกระจกมองข้างขวา,ฝาปิดสเกิร์ต,พลาสติกรองสกรู,ยางกันฝุ่นเบคร,สกรู,หมุดยึด,หมุดยึดกันชน,หัวเทียน
bill_number,,,,,,,,,,,,,,
A22-00024,0,0,0,0,0,0,0,0,0,0,0,0,0,1
A22-00033,1,0,0,0,0,0,0,0,0,0,0,0,0,0
A22-00034,0,0,0,0,0,0,0,0,0,1,0,0,0,0
A22-00052,0,0,0,0,0,0,0,0,0,0,1,0,0,0
A22-00120,0,0,0,0,0,1,0,0,0,0,0,0,0,0
A22-00138,0,0,0,0,0,0,0,0,0,0,0,1,0,0
A22-00160,0,0,0,0,0,0,0,0,0,0,0,0,1,0
A22-00161,0,0,0,0,0,0,0,0,0,0,0,0,1,0
A22-00165,0,0,0,0,0,0,0,0,0,0,0,1,0,0


# แปลงข้อมูลไปเป็น bool เพื่อให้การจัดเก็บข้อมูลนั้นน้อยลง

In [8]:
basket_sets=basket_sets.astype(pd.SparseDtype("bool"))

# ทำการหาค่า sopport 
- การหาค่า sopport เพื่อดูว่า สินค้าชิ้นไหนที่ลูกค้ามีการสั่งชื้อมากที่สุดหรือบ่อยที่สุดหน้าร้าน
- โดยเริ่มจากค่า 0.01ก่อน หรือ 1% ของสินค้าทั้งหมด

In [9]:
frequent_itemsets = apriori(basket_sets, min_support=0.01,use_colnames=True)
frequent_itemsets.sort_values('support',ascending=False)

,support,itemsets
11,0.285714,(หมุดยึด)
13,0.214286,(หัวเทียน)
12,0.107143,(หมุดยึดกันชน)
6,0.071429,(ฝาครอบกระจกมองข้างขวา)
0,0.035714,(คลิ๊บยางรองกันฝุ่น)
1,0.035714,(คลิ๊บล็อค)
2,0.035714,(คลิ๊บแผงใต้เครื่อง)
3,0.035714,(น๊อต ยึดหน้าแกน)
4,0.035714,(บู๊ชหูแหนบ)
5,0.035714,(ประเก็นหัวเทียน)


# ลองหาค่าของ lift ดูว่า มีสินค้าตัวไหนที่ลูกค้ามักจะชื้อคู่กันบาง ?
- คำตอบคือ ไม่มีสินค้าตัวไหนที่ชื้อคู่กันตลอด เพราะมีข้อมูลแค่ 1 ปีเท่านั้นอาจจะพบมากกว่านี้ถ้ามีข้อมูลมากขึ้น
- เนื่องจากข้อมูลนี้เป็นข้อมูลอะไหล่รถยนต์ที่เป็นบิลเงินสดหน้าร้าน
- อาจจะตีความได้ว่า ลูกค้าส่วนใหญ่มักชื้อสินค้าเป็น สินค้าอย่างเดียว เช่น คลิ๊บแผงใต้เครื่อง ลูกค้าอาจจะชื้ออย่างเดียวแต่หลายชิ้น
- ลูกค้าชื้อของตามอาการของรถ เช่น ต้องการเปลี่ยนหัวเทียน ก็ชื้อแต่หัวเทียนไปเปลี่ยน โดยที่ไม่ชื้ออย่างอื่นคู่ไปด้วย
- ทั้งนี้ต้องตรวจสอบค่าอื่นๆด้วยเพื่อความชัดเจนว่าลูกค้าของบริษัทนั้น มีพฤติกรรมที่ชื้อสินค้าเป็น items ชิ้นเดียวจริงหรือไม่ โดยจะต้องใช้ค่า 3 ตัว
- support lift confidence
- support คือจำนวนการชื้อ เทียบกับจำนวน สินค้าทั้งหมด
- lift คือค่าที่ได้จากการหาความสัมพันธ์ว่ามีสินค้าไหนที่มักจะชื้อไปด้วยกัน
- confidence เป็นค่าเปรียบเทียบกันของทั้ง 2 items

In [10]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(หมุดยึด),(พลาสติกรองสกรู),0.285714,0.035714,0.035714,0.125,3.5,0.02551,1.102041
1,(พลาสติกรองสกรู),(หมุดยึด),0.035714,0.285714,0.035714,1.000,3.5,0.02551,inf


In [11]:
rules = association_rules(frequent_itemsets, metric = "support", min_threshold = 0.02)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(หมุดยึด),(พลาสติกรองสกรู),0.285714,0.035714,0.035714,0.125,3.5,0.02551,1.102041
1,(พลาสติกรองสกรู),(หมุดยึด),0.035714,0.285714,0.035714,1.000,3.5,0.02551,inf


In [12]:
rules = association_rules(frequent_itemsets, metric = "confidence", min_threshold = 1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(พลาสติกรองสกรู),(หมุดยึด),0.035714,0.285714,0.035714,1.0,3.5,0.02551,inf


# จากการ ดูค่า support lift confidence
- พบว่าลูกค้ามีพฤติกรรมการชื้อของที่เป็นคู่เหมือนกัน
- โดยอะไหล่ที่มีความสัมพันธ์กัน คือ พลาสติกรองสกรู กับ หมุดยึด
- แสดงว่าอะไหล่ 2 อย่างนี้จ่ายคู่กันบ่อยที่สุดในปี 2022

# แต่เนื่องจากว่า ลูกค้าส่วนใหญ่ชื้อสินค้าเป็น items เดียว เลยสงสัยว่าอะไรบางที่ลูกค้านิยมชื้อมากกที่สุด เมื่อเทียบกับ ข้อมูลทั้งหมด

In [13]:
frequent_itemsets = apriori(basket_sets, min_support = 0.1, use_colnames = True)
frequent_itemsets['support_n']=frequent_itemsets['support']*len(basket_sets)
frequent_itemsets.sort_values('support', ascending=False)

,support,itemsets,support_n
0,0.285714,(หมุดยึด),8.0
2,0.214286,(หัวเทียน),6.0
1,0.107143,(หมุดยึดกันชน),3.0


# จากการคำนวณพบว่า
- สินค้าบิลเงินสดของสินค้าในปี 2022 พบว่าอะไหล่ที่มีการชื้อบ่อยที่สุด 3 ลำดับแรกที่มี %การชื้อที่สูง
- 1 หมุดยึด มี %การชื้ออยู่ที่ 80 % ของสินค้าทั้งหมด
- 2 หัวเทียน มี %การชื้ออยู่ที่ 60 % ของสินค้าทั้งหมด
- 3 หมุดยึดกันชน มี %การชื้ออยู่ที่ 30 % ของสินค้าทั้งหมด

In [14]:
basket_sets.sum()/len(basket_sets)

part_name
คลิ๊บยางรองกันฝุ่น       0.035714
คลิ๊บล็อค                0.035714
คลิ๊บแผงใต้เครื่อง       0.035714
น๊อต ยึดหน้าแกน          0.035714
บู๊ชหูแหนบ               0.035714
ประเก็นหัวเทียน          0.035714
ฝาครอบกระจกมองข้างขวา    0.071429
ฝาปิดสเกิร์ต             0.035714
พลาสติกรองสกรู           0.035714
ยางกันฝุ่นเบคร           0.035714
สกรู                     0.035714
หมุดยึด                  0.285714
หมุดยึดกันชน             0.107143
หัวเทียน                 0.214286
dtype: float64

# สรุปการวิเคราะห์ Market Basket Analysis 
- พบว่าลูกค้าส่วนใหญ่มีพฤติกรรมการชื้อสินค้าเป็น ชิ้นเดียวต่อบิล
- จากการวิเคราะห์จะเห็นว่าลูกค้าส่วนใหญ่มักชื้ออะไหล่ชิ้นเล็กๆที่สามารถเปลี่ยนเองได้
# ข้อเสนอแนะ
- ถ้าลูกค้ามาชื้อของหน้าร้าน ไม่ต้องทำการลดราคาสินค้าในกรณีที่ ราคาสินค้าไม่ถึง 500บาท เพราะจากการดูข้อมูลแล้ว
- ลูกค้ามักชื้อไปเปลี่ยนเอง เพราะ ไม่ต้องการเปิด job เสียค่าแรง ดังนั้น ถ้าราคาอะไหล่ ไม่ถึง 500 บาท ไม่ต้องทำการลดราคาทุกกรณี
- หรือทำการเสนอให้ลูกค้าเปิด job เพื่อรับการประกันอะไหล่ 1 ปี 20000 กิโล